In [ ]:
Api_Key = "_J=IUwKV(hg^Upo5NXsaim*MHB_h4xbQlJf&hmGN"

Group_Ids = "-5950303211111682806"

Insights_ApiUrl = "https://insights.us.mixtelematics.com"

Refresh_WindowInMonths = 1   

In [ ]:
from datetime import datetime, timedelta
import math
import requests
import pandas as pd
from numpy import int64
from numpy import float64

In [ ]:
# Variáveis
InsightsApiUrl = Insights_ApiUrl # Substitua pela URL base real
ApiKey = Api_Key
GroupIds = Group_Ids  # Pode ser múltiplos IDs separados por vírgula se necessário
RefreshWindowInMonths = Refresh_WindowInMonths

In [ ]:
# === 1. CALCULA INTERVALO DE DATAS ===
to_date = datetime.now().date()
from_date = to_date - timedelta(days=RefreshWindowInMonths * 30)
date_diff = (to_date - from_date).days
intervalos = [i for i in range(0, math.floor(date_diff / 30) + 1)]

In [ ]:
# === 2. FUNÇÃO PARA FORMATAR DATA ===
def format_date(date):
    return date.strftime("%Y%m%d")

In [ ]:
# === 3. GERA LISTA DE PERÍODOS MENSAL ===
periodos = []
for i in intervalos:
    start_date = from_date + timedelta(days=i * 30)
    end_date = start_date + timedelta(days=29)
    periodos.append({
        "start": start_date,
        "end": end_date,
        "start_fmt": format_date(start_date),
        "end_fmt": format_date(end_date)
    })

In [ ]:
# === 4. CONSULTA A API PARA CADA PERÍODO ===
dados_coletados = []
for p in periodos:
    url = f"{InsightsApiUrl}/api/aggregates/events/assets/{GroupIds}?startDate={p['start_fmt']}&endDate={p['end_fmt']}"
    headers = {"X-API-KEY": ApiKey}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        dados_coletados.extend(response.json())
    else:
        print(f"Erro {response.status_code} na chamada entre {p['start_fmt']} e {p['end_fmt']}")

In [ ]:
print(dados_coletados[0].keys())

In [ ]:
# === 5. CRIA DATAFRAME E AJUSTA TIPOS ===

df = pd.DataFrame(dados_coletados)
if not df.empty:
    df = df.astype({
        "DateKey": "Int64", 
        "GroupId": "Int64", 
        "SiteId": "Int64", 
        "AssetId": "Int64", 
        "TotalEventOccurrences": "Int64", 
        "EventTypeId": "Int64", 
        "MinEventValue":"float64", 
        "MaxEventValue":"float64", 
        "TotalEventValue":"float64", 
        "MinEventDuration":"float64", 
        "MaxEventDuration":"float64", 
        "TotalEventDuration":"float64"
    }, errors='ignore')

In [ ]:
# === 6. CRIA COLUNA COMPOSTA ===
df["CompositeGroupEventTypeId"] = df["GroupId"].astype(str) + ":" + df["EventTypeId"].astype(str)
df["CompositeGroupEventTypeId"] = df["CompositeGroupEventTypeId"].where(df["CompositeGroupEventTypeId"].notnull(), None)

# === 7. MOSTRA OS PRIMEIROS REGISTROS ===
df.head()